## Save nbar
Due to a recent PR, we now save the number density of every summary in the group attributes. This notebook goes back and saves `nbar` for all older diagnostics files.

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

import os
from os.path import join
import pickle
import json
import h5py
import matplotlib as mpl
import matplotlib.pyplot as plt
# mpl.style.use('../../style.mcstyle')   # noqa
import numpy as np
import torch
import seaborn as sns
import pandas as pd
import warnings
from tqdm import tqdm

# Suppress warnings
warnings.filterwarnings('ignore')

In [7]:
wdir = '/anvil/scratch/x-mho1/cmass-ili'
nbody = 'abacus'
sim = 'correct'
L, N = 1000, 128
L, N = 2000, 256

suitepath = join(wdir, nbody, sim, f'L{L}-N{N}')

tracer = 'halos'

In [ ]:
def compute_nbar(Ngals, L):
    return Ngals / L**3


def compute_log10nbar(Ngals, L):
    return np.log10(Ngals) - 3 * np.log10(L)


def get_lc_L(cap):
    if cap == 'ngc':
        L = 3500
    elif cap == 'sgc':
        L = 2750
    elif cap == 'mtng':
        L = 2000
    elif cap == 'simbig':
        L = 2000
    else:
        raise ValueError
    return L


def process_file(ingroup, outgroup, Nmin, L):
    key = 'pos' if 'pos' in ingroup.keys() else 'ra'
    Ngals = min(len(ingroup[key]), Nmin)
    nbar = compute_nbar(Ngals, L)
    log10nbar = compute_log10nbar(Ngals, L)
    print(nbar, log10nbar)
    outgroup.attrs['nbar'] = nbar
    outgroup.attrs['log10nbar'] = log10nbar


def process_halogalaxy_diag(tracerfile, diagfile, Nmin, L):
    with h5py.File(tracerfile, 'r') as g, h5py.File(diagfile, 'a') as f:
        for key in f.keys():
            process_file(g[key], f[key], Nmin, L)


def process_lightcone_diag(tracerfile, diagfile, Nmin, L):
    with h5py.File(tracerfile, 'r') as g, h5py.File(diagfile, 'a') as f:
        process_file(g, f, Nmin, L)


def process_lhid(lhid, suitepath, tracer, Nmin, L):
    tracerpath = join(suitepath, lhid, tracer)
    diagpath = join(suitepath, lhid, 'diag', tracer)
    if tracer == 'halos':
        tracerpath += '.h5'
        diagpath += '.h5'
        if not os.path.exists(diagpath):
            return
        process_halogalaxy_diag(tracerpath, diagpath, Nmin, L)
    else:  # galaxies and lightcone
        if not os.path.exists(diagpath):
            return
        for f in os.listdir(diagpath):
            diagfile = join(diagpath, f)
            tracerfile = join(tracerpath, f)
            if tracer == 'galaxies':
                process_halogalaxy_diag(tracerfile, diagfile, Nmin, L)
            elif 'lightcone' in tracer:
                L = get_lc_L(tracer.split('_')[0])
                process_lightcone_diag(tracerfile, diagfile, Nmin, L)


nbar_min = np.inf
Nmin = nbar_min * L**3

for lhid in tqdm(os.listdir(suitepath)):
    process_lhid(lhid, suitepath, tracer, Nmin, L)

100%|██████████| 52/52 [00:00<00:00, 59.94it/s]
